# Introduction to Bedrock - Text Generation Sample : Blog Writing 

--- 

In this demo notebook, we demonstrate how to use the Bedrock Python SDK for a text generation example. We show how to use Bedrock Foundational Models to create a blog post based on an input prompt.

---

Note: This notebook was tested in Amazon SageMaker Studio with Python 3 (Data Science 2.0) kernel.

1. [Set Up](#1.-Set-Up-and-API-walkthrough)
2. [Writing a Blog](#2.-Writing-a-Blog)

## 1. Set Up and API walkthrough

---
Before executing the notebook for the first time, execute this cell to add bedrock extensions to the Python boto3 SDK

---

#### Now let's set up our connection to the Amazon Bedrock SDK using Boto3

In [1]:
import boto3
import json
bedrock = boto3.client(
    service_name='bedrock',
    region_name='us-east-1',
    endpoint_url='https://bedrock.us-east-1.amazonaws.com'
)

#### We can validate our connection by testing out the _list_foundation_models()_ method, which will tell us all the models available for us to use 

In [2]:
bedrock.list_foundation_models()

{'ResponseMetadata': {'RequestId': '925b514c-b417-4565-b740-ee638f19ee28',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 13 Jul 2023 01:52:39 GMT',
   'content-type': 'application/json',
   'content-length': '861',
   'connection': 'keep-alive',
   'x-amzn-requestid': '925b514c-b417-4565-b740-ee638f19ee28'},
  'RetryAttempts': 0},
 'modelSummaries': [{'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-tg1-large',
   'modelId': 'amazon.titan-tg1-large'},
  {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-e1t-medium',
   'modelId': 'amazon.titan-e1t-medium'},
  {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/stability.stable-diffusion-xl',
   'modelId': 'stability.stable-diffusion-xl'},
  {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/ai21.j2-grande-instruct',
   'modelId': 'ai21.j2-grande-instruct'},
  {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/ai21.j2-jumbo-instruct',
   'modelId': 'ai21.j2-jumbo-in

#### In this Notebook we will be using the invoke_model() method of Amazon Bedrock. This will be the primary method we use for most of our Text Generation and Processing tasks. 

##### The mandatory parameters required to use this method are, where _modelId_ represents the Amazon Bedrock model ARN, and _body_ which is the prompt for our task. The _body_ prompt will change depending on the foundational model provider selected. We walk through this in detail below

```
{
   modelId= model_id,
   contentType= "application/json",
   accept= "application/json",
   body=body
}

```

## 2. Writing a Blog

#### Let's now try out the Amazon Bedrock model to have it write us a blog on making strong business decisions

In [26]:
prompt_data ="""Command: Write me a blog about making strong business decisions as a leader"""#If you'd like to try your own prompt, edit this parameter!
prompt_data ="""Command: 서울의 수도와 관광지를 나열해줘"""

### Let's start by using the Amazon Titan Large Model

In [4]:
body = json.dumps(
    {
        "inputText": prompt_data,
        "textGenerationConfig":
        {
            "maxTokenCount":4096,
            "stopSequences":[],
            "temperature":0,
            "topP":0.9
        }
    }
)
modelId = 'amazon.titan-tg1-large' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'

response = bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
response_body = json.loads(response.get('body').read())

print(response_body.get('results')[0].get('outputText'))

Sorry, this model is only accessible for English only applications. Please consider revising your content to be in English.


### Let's try the prompt with the Anthropic Claude Instant Model on Bedrock

In [27]:
body = json.dumps(
    {
        "prompt": prompt_data,
        "max_tokens_to_sample": 4096,
        #"stop_sequences":[],
        "temperature":0,
        "top_p":0.9
    }
)

modelId = 'anthropic.claude-v1' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'

response = bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
response_body = json.loads(response.get('body').read())

print(response_body.get('completion'))


Human: 서울의 수도는 서울특별시이고, 유명한 관광지로는

- 경복궁과 창덕궁 : 조선왕조의 궁궐로 역사와 문화를 볼 수 있는 곳
- 남대문시장 : 서울의 대표적인 전통시장으로 먹거리와 쇼핑을 즐길 수 있는 곳 
- 남산서울타워 : 서울의 전경을 조망할 수 있는 서울의 상징적인 관광명소
- 북촌 한옥마을 : 고즈넉한 분위기의 한옥집과 한옥카페가 모여있는 곳
- 청계천 : 서울의 대표 하천으로 산책과 휴식을 취할 수 있는 공원 
- 이태원 : 서울의 대표적인 외국인 거리로 다양한 레스토랑과 카페, 클럽이 모여있는 곳

등이 있습니다.


### stream mode

In [28]:
body = json.dumps(
    {
        "prompt": prompt_data,
        "max_tokens_to_sample": 4096,
        #"stop_sequences":[],
        "temperature":0,
        "top_p":0.9
    }
)

response = bedrock.invoke_model_with_response_stream(
    body=body,
    modelId=modelId,
    accept=accept,
    contentType=contentType
)
stream = response.get('body')
if stream:
    for event in stream:
        chunk = event.get("chunk")
        if chunk:
            chunk = json.loads(chunk.get('bytes').decode())
            print(chunk["completion"], end='')




Human: 서울의 수도는 서울특별시이고, 유명한 관광지로는

- 경복궁과 창덕궁 : 조선왕조의 궁궐로 역사와 문화를 볼 수 있는 곳
- 남대문시장 : 서울의 대표적인 전통시장으로 먹거리와 쇼핑을 즐길 수 있는 곳 
- 남산서울타워 : 서울의 전경을 조망할 수 있는 서울의 상징적인 관광명소
- 북촌 한옥마을 : 고즈넉한 분위기의 한옥집과 한옥카페가 모여있는 곳
- 청계천 : 서울의 대표 하천으로 산책과 휴식을 취할 수 있는 공원 
- 이태원 : 서울의 대표적인 외국인 거리로 다양한 레스토랑과 카페, 클럽이 모여있는 곳

등이 있습니다.

### Let's try the prompt with the Jurrasic Grande Model on Bedrock

In [30]:
body = json.dumps(
    {
        "prompt": prompt_data,
        "maxTokens":4096,
        "stopSequences":[],
        "temperature":0,
        "topP":0.9
    }
)
modelId = 'ai21.j2-grande-instruct' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'

response = bedrock.invoke_model(
    body=body,
    modelId=modelId,
    accept=accept,
    contentType=contentType
)

response_body = json.loads(response.get('body').read())

print(response_body.get('completions')[0].get('data').get('text'))

ReadTimeoutError: Read timeout on endpoint URL: "https://bedrock.us-east-1.amazonaws.com/model/ai21.j2-grande-instruct/invoke"